In [109]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import re
from bs4 import BeautifulSoup
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer
 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ayan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ayan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ayan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [106]:
# ! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

## Read Data

In [3]:
df = pd.read_csv('./amazon_reviews_us_Beauty_v1_00.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)


C:\Users\Ayan\AppData\Local\Temp\ipykernel_5428\3693539848.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('./amazon_reviews_us_Beauty_v1_00.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)
C:\Users\Ayan\AppData\Local\Temp\ipykernel_5428\3693539848.py:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('./amazon_reviews_us_Beauty_v1_00.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)
C:\Users\Ayan\AppData\Local\Temp\ipykernel_5428\3693539848.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./amazon_reviews_us_Beauty_v1_00.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)


## Keep Reviews and Ratings

In [4]:
df = df[['star_rating', 'review_body']]

# let ratings with the values of 1 and 2 form class 1, ratings with the value of 3 form class 2, and ratings with the values of 4 and 5 form class 3

In [5]:
class_one = df[(df['star_rating']==1) | (df['star_rating']==2)]
class_two = df[df['star_rating']==3]
class_three = df[(df['star_rating']==4) | (df['star_rating']==5)]
print(len(class_one), len(class_two), len(class_three))

588295 324095 3198869


 ## We form three classes and select 20000 reviews randomly from each class.



In [95]:
class_one = class_one.sample(n=20000, random_state=100)
class_two = class_two.sample(n=20000, random_state=100)
class_three = class_three.sample(n=20000, random_state=100)
dataset = pd.concat([class_one, class_two, class_three])
dataset.reset_index(drop=True)
train = dataset.sample(frac=0.8, random_state=100)
test = dataset.drop(train.index)
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)
print(len(train), len(test))

48000 12000


# Data Cleaning



# convert all reviews into lowercase

In [96]:
avg_len_before = (train['review_body'].str.len().sum() + test['review_body'].str.len().sum())/60000

#Covert all reviews to lower case
train['review_body'] = train['review_body'].str.lower()
test['review_body'] = test['review_body'].str.lower()

# remove the HTML and URLs from the reviews

In [97]:
# URL Remover code (Ref:https://stackoverflow.com/questions/51994254/removing-url-from-a-column-in-pandas-dataframe)
train['review_body'] = train['review_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
test['review_body'] = test['review_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

def html_tag_remover(review):
    soup = BeautifulSoup(review, 'html.parser')
    review = soup.get_text()
    return review  

train['review_body'] = train['review_body'].apply(lambda review: html_tag_remover(review))
test['review_body'] = test['review_body'].apply(lambda review: html_tag_remover(review))

C:\Users\Ayan\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


# remove non-alphabetical characters

In [98]:
train['review_body'] = train['review_body'].apply(lambda review: re.sub('[^a-zA-Z]+',' ', review))
test['review_body'] = test['review_body'].apply(lambda review: re.sub('[^a-zA-Z]+',' ', review))


# remove extra spaces

In [99]:
train['review_body'] = train['review_body'].apply(lambda review: re.sub(' +', ' ', review))
test['review_body'] = test['review_body'].apply(lambda review: re.sub(' +', ' ', review))

# perform contractions on the reviews

In [101]:
def expand_contractions(review):
    review = contractions.fix(review)
    return review

train['review_body'] = train['review_body'].apply(lambda review: expand_contractions(review))
test['review_body'] = test['review_body'].apply(lambda review: expand_contractions(review))
avg_len_after = (train['review_body'].str.len().sum() + test['review_body'].str.len().sum())/60000
print(avg_len_before, avg_len_after)                                                                      

287.3779333333333 276.02595


# Pre-processing

## remove the stop words 

In [102]:
avg_len_before_prepro = (train['review_body'].str.len().sum() + test['review_body'].str.len().sum())/60000
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stopwords(review):
    stop_words_english = set(stopwords.words('english'))
    review_word_tokens = word_tokenize(review)
    filtered_review = [word for word in review_word_tokens if not word in stop_words_english]
    return filtered_review

train['review_body'] = train['review_body'].apply(lambda review: remove_stopwords(review))
test['review_body'] = test['review_body'].apply(lambda review: remove_stopwords(review))


276.02595


## perform lemmatization  

In [104]:
from nltk.stem import WordNetLemmatizer

def review_lemmatize(review):
    lemmatizer = WordNetLemmatizer()
    lemmatized_review = [lemmatizer.lemmatize(word) for word in review]
    return ' '.join(lemmatized_review)    

train['review_body'] = train['review_body'].apply(lambda review: review_lemmatize(review))
test['review_body'] = test['review_body'].apply(lambda review: review_lemmatize(review))

avg_len_after_prepro = (train['review_body'].str.len().sum() + test['review_body'].str.len().sum())/60000
print(avg_len_before_prepro, avg_len_after_prepro)


276.02595 165.65626666666665


# TF-IDF Feature Extraction

In [110]:
train_reviews = list(train['review_body'])

['followed direction waited hour really casting judgement cover tattoo yes look like skin really first pack come one skin tone big sheet need two sheet cover tattoo want use tone lucky bought one pack second yes crack wrinkle showed like review mentioned eventually sort look like skin aspect however cover apparently enough look natural looked like wearing tattoo shaped bandaid seems blending makeup required', 'quality good', 'research fact found external use pure product fault bought cheap', 'product say uncomfortable use worth plastic cone awkward size people hurt neck minute resting face like electric face brush great exfoliating', 'used calico nail bandage year unable find store went amazon see could find product package looked internet used thick would stay place nail happy']


# Perceptron

# SVM

# Logistic Regression

# Naive Bayes